# Digital Twin PBPK-QSP Model

This notebook simulates drug pharmacokinetics (PK) and quantitative systems pharmacology (QSP) using a physiologically-based pharmacokinetic (PBPK) model for a human "digital twin." 

**Purpose:**  
- Allow interactive exploration of how physiological and drug-specific parameters affect drug disposition.
- Enable extension to new compartments or QSP mechanisms via configuration.

**References:**  
- Rowland, M., & Tozer, T. N. (2011). *Clinical Pharmacokinetics and Pharmacodynamics: Concepts and Applications* (4th ed.). Lippincott Williams & Wilkins.
- Jones, H. M., & Rowland-Yeo, K. (2013). Basic concepts in physiologically based pharmacokinetic modeling in drug discovery and development. *CPT: Pharmacometrics & Systems Pharmacology*, 2(8), e63.


In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from dataclasses import dataclass
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import yaml
import json
from IPython.display import display

# Utility for PK metrics
def compute_pk_metrics(df, comp='Blood'):
    conc = df[comp]
    time = df['Time_h']
    cmax = np.max(conc)
    tmax = time[np.argmax(conc)]
    auc = np.trapz(conc, time)
    return {'Cmax': cmax, 'Tmax': tmax, 'AUC': auc}

In [ ]:
@dataclass
class Physiology:
    """Physiological parameters for a human digital twin."""
    weight_kg: float = 70.0
    V_blood: float = 5.0
    V_liver: float = 1.8
    Q_liver: float = 90.0
    V_muscle: float = 29.0
    Q_muscle: float = 450.0
    V_fat: float = 18.0
    Q_fat: float = 30.0
    CLint_liver: float = 20.0

@dataclass
class Compound:
    """Compound-specific PBPK properties."""
    name: str = "DrugX"
    Kp_liver: float = 10.0
    Kp_muscle: float = 3.0
    Kp_fat: float = 50.0
    k_abs: float = 1.0

@dataclass
class DosingEvent:
    """Defines a dose event: bolus, infusion, or oral."""
    type: str      # 'iv_bolus', 'iv_infusion', 'oral'
    amount: float  # mg
    time: float    # h
    duration: float = 0.0  # infusion duration in h

class PBPKQSPSimulator:
    """
    Simulates PBPK-QSP dynamics for a given physiology, compound, and dosing regimen.
    """
    def __init__(self, phys: Physiology, cmpd: Compound, qsp_params=None):
        self.phys = phys
        self.cmpd = cmpd
        self.qsp = qsp_params  # tuple(kon, koff, Rtot, kprod, kdeg) or None
        self.names = ['Gut','Blood','Liver','Muscle','Fat']
        self.vols = [1.0, phys.V_blood, phys.V_liver, phys.V_muscle, phys.V_fat]
        self.flows = [0.0, 0.0, phys.Q_liver, phys.Q_muscle, phys.Q_fat]
        self.kps   = [1.0, 1.0, cmpd.Kp_liver, cmpd.Kp_muscle, cmpd.Kp_fat]
        self.clint = [0.0, 0.0, phys.CLint_liver, 0.0, 0.0]
        self.n_pk = len(self.names)
        self.n_qsp = 3 if qsp_params else 0

    def odes(self, t, y, events):
        # 1) dosing injections (mg/h)
        inj = np.zeros(self.n_pk)
        for ev in events:
            if ev.type=='iv_bolus' and abs(t-ev.time)<self.dt/2:
                inj[1] += ev.amount/self.dt
            elif ev.type=='iv_infusion' and ev.time <= t < ev.time+ev.duration:
                inj[1] += ev.amount/ev.duration
            elif ev.type=='oral' and abs(t-ev.time)<self.dt/2:
                inj[0] += ev.amount/self.dt

        # 2) PBPK states and derivatives
        Agut, Ab, Aliv, Amusc, Afat = y[:5]
        dAgut = -self.cmpd.k_abs*Agut + inj[0]
        Cb = Ab/self.vols[1]
        dAb = self.cmpd.k_abs*Agut + inj[1]
        dAliv = dAmusc = dAfat = 0.0
        for idx, (vol, flow, kp, cl) in enumerate(zip(self.vols, self.flows, self.kps, self.clint)):
            if idx<2: continue
            Ci = y[idx]/vol
            flux = flow*(Cb - Ci/kp)
            dAb   -= flux
            if idx==2: dAliv = flux - cl*Ci
            elif idx==3: dAmusc = flux
            elif idx==4: dAfat  = flux

        dPK = [dAgut, dAb, dAliv, dAmusc, dAfat]

        # 3) QSP if provided
        if self.qsp:
            kon, koff, Rtot, kprod, kdeg = self.qsp
            Rf, Rc, M = y[5:8]
            bind = kon*Cb*Rf - koff*Rc
            dRf = -bind
            dRc = bind
            dM  = kprod*Rc - kdeg*M
            return dPK + [dRf, dRc, dM]

        return dPK

    def simulate(self, events, t_end=24.0, dt=0.1):
        """
        Simulate the PBPK-QSP system.
        Returns: DataFrame with time and compartment amounts.
        """
        y0 = [0.0]*self.n_pk
        if self.qsp:
            y0 += [self.qsp[2], 0.0, 0.0]  # Rf, Rc, M
        self.dt = dt
        t_eval = np.arange(0, t_end+dt, dt)
        sol = solve_ivp(
            fun=lambda t,y: self.odes(t,y,events),
            t_span=(0, t_end), y0=y0, t_eval=t_eval, method='RK45'
        )
        cols = self.names.copy()
        if self.qsp:
            cols += ['Free_Receptor','Drug_Receptor_Complex','Biomarker']
        df = pd.DataFrame(sol.y.T, columns=cols)
        df.insert(0, 'Time_h', sol.t)
        return df

In [ ]:
# Load YAML config
def load_config(path='pbpk_config.yaml'):
    with open(path, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

cfg = load_config()

# Widgets for user parameterization
phys_widget = widgets.interactive(
    lambda weight_kg, V_blood, V_liver, Q_liver, V_muscle, Q_muscle, V_fat, Q_fat, CLint_liver: Physiology(
        weight_kg, V_blood, V_liver, Q_liver, V_muscle, Q_muscle, V_fat, Q_fat, CLint_liver
    ),
    weight_kg=cfg['physiology']['weight_kg'],
    V_blood=cfg['physiology']['V_blood'],
    V_liver=cfg['physiology']['V_liver'],
    Q_liver=cfg['physiology']['Q_liver'],
    V_muscle=cfg['physiology']['V_muscle'],
    Q_muscle=cfg['physiology']['Q_muscle'],
    V_fat=cfg['physiology']['V_fat'],
    Q_fat=cfg['physiology']['Q_fat'],
    CLint_liver=cfg['physiology']['CLint_liver']
)

cmpd_widget = widgets.interactive(
    lambda name, Kp_liver, Kp_muscle, Kp_fat, k_abs: Compound(
        name, Kp_liver, Kp_muscle, Kp_fat, k_abs
    ),
    name=cfg['compound']['name'],
    Kp_liver=cfg['compound']['Kp_liver'],
    Kp_muscle=cfg['compound']['Kp_muscle'],
    Kp_fat=cfg['compound']['Kp_fat'],
    k_abs=cfg['compound']['k_abs']
)

# QSP parameters
qsp_widget = widgets.interactive(
    lambda kon, koff, Rtot, kprod, kdeg: (kon, koff, Rtot, kprod, kdeg),
    kon=cfg['qsp']['kon'],
    koff=cfg['qsp']['koff'],
    Rtot=cfg['qsp']['Rtot'],
    kprod=cfg['qsp']['kprod'],
    kdeg=cfg['qsp']['kdeg']
)

display(widgets.HTML("<b>Adjust Physiology Parameters:</b>"))
display(phys_widget)
display(widgets.HTML("<b>Adjust Compound Parameters:</b>"))
display(cmpd_widget)
display(widgets.HTML("<b>Adjust QSP Parameters:</b>"))
display(qsp_widget)

In [ ]:
# Dosing events from config
dosing_events = [DosingEvent(**d) for d in cfg['dosing']]

# Optionally, you could add widgets for dosing, but for brevity, we use config here.
display(pd.DataFrame(cfg['dosing']))

In [ ]:
# Get user-selected parameters
phys = phys_widget.result
cmpd = cmpd_widget.result
qsp = qsp_widget.result

sim = PBPKQSPSimulator(phys, cmpd, qsp_params=qsp)
df = sim.simulate(dosing_events, t_end=24.0, dt=0.1)

display(df.head())

# Plot concentrations in each compartment
plt.figure(figsize=(10,6))
for col in sim.names:
    plt.plot(df['Time_h'], df[col], label=col)
plt.xlabel('Time (h)')
plt.ylabel('Drug Amount (mg)')
plt.title('Drug Amount in Compartments Over Time')
plt.legend()
plt.show()

# PK metrics for Blood
pk = compute_pk_metrics(df, comp='Blood')
print("PK Metrics (Blood):")
for k, v in pk.items():
    print(f"{k}: {v:.2f}")

## Extending the Model

- To add new compartments or QSP mechanisms, edit the `pbpk_config.yaml` and update the `PBPKQSPSimulator` class accordingly.
- For advanced users: You can further modularize the code by moving classes/functions into `.py` modules and importing them.
- For more advanced visualization, consider using `plotly` for interactive plots.

---

**You now have an interactive, well-documented, and extensible PBPK-QSP notebook!**